Lecture week 5
==============



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sn
from scipy import optimize
import pymc3 as pm
import statsmodels.api as sm # check the error that cannot import name 'factorial' in from scipy.misc import factorial
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import tensorflow as tf
import altair as alt
# from linearmodels.iv import IV2SLS
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
import arviz as az

import theano
import covid19pandas as cod
from country_codes import eurostat_dictionary
import eurostat
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

In [ ]:
EU_countries = ['Belgium', 'Bulgaria', 'Czechia', 'Denmark',
'Germany (until 1990 former territory of the FRG)', 'Germany','Estonia',
'Ireland', 'Greece', 'Spain', 'France', 'Croatia', 'Italy',
'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Malta',
'Netherlands', 'Austria', 'Poland', 'Portugal', 'Romania',
'Slovenia', 'Slovakia', 'Finland', 'Sweden', 'United Kingdom',
'Iceland', 'Liechtenstein', 'Norway', 'Switzerland',
'Bosnia and Herzegovina']

## Mortality



The [Eurostat website](https://ec.europa.eu/eurostat/data/database?node_code=hlth_cd_apr) has a browser where you can look for data. Here we are looking for data on mortality. You can click on the link to the data browser to see the [details of the variable](https://ec.europa.eu/eurostat/databrowser/view/hlth_cd_apr/default/table?lang=en). At the top-left of the screen you can see the name of the variable in the line &ldquo;`online data code: HLTH_CD_APR`&rdquo;. The name of this variable we use below in the `get_data_df` method.

So we call this method and collect the information in the dataframe `df`. Then we check what `df` looks like:



In [ ]:
df = eurostat.get_data_df('hlth_cd_apr')
df.head()

We would like to get our data into the following shape:

| country|year|sex|Preventable  mortality|Treatable  mortality|
|---|---|---|---|---|
| Austria|2011|F|95.29|67.41|
| Austria|2011|M|248.50|96.86|
| Austria|2012|F|96.16|69.72|
| Austria|2012|M|252.28|91.45|
| Austria|2013|F|93.22|66.84|

Hence we need to use pandas to get from our initial `df` to a dataframe like this table.



In [ ]:
df.rename({'geo\\time':'geo'},inplace=True,axis=1)
df['country'] = df['geo'].replace(eurostat_dictionary)
df.head()

In [ ]:
df = df[df.country.isin(EU_countries) & (df.sex.isin(["M","F"]) ) & (df.mortalit.isin(["PRVT","TRT"])) \
        & (df.unit == "RT") & (df.icd10 == "TOTAL")]
df.drop(["unit","icd10","geo"],axis=1,inplace=True)
df.head()

Now we want to &ldquo;move&rdquo; the years into rows.

-   Pandas&rsquo; [melt method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) helps us to get the years in a column `year`
-   the mortality rates in a column `rate`
-   columns 2011-2017 (`np.arange(2011,2018)` in the code) become row values in the column `year`
-   this column `year` together with the identifier variables uniquely identifies a row
-   values for PRVT and TRT are given in the column `rate`.



In [ ]:
df = pd.melt(df,id_vars=['country','sex','mortalit'],
                        value_vars=np.arange(2011,2018),
                        var_name='year',
                        value_name='rate')
df.head()

-   we want to have two columns (i.e. two variables); one corresponds to PRVT, the other to TRT
-   we use [unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html): put the identifying columns in an order such that the last column refers to `mortalit` which contains the two values PRVT and TRT
-   these columns become the index of the dataframe
-   unstack the dataframe on the last column of the index, which is the default value of `unstack()`
-   pivots the column `mortalit` into two separate columns PRVT and TRT.



In [ ]:
df.set_index(['country','year','sex','mortalit'],inplace=True)
df = df.unstack()
df.head()

-   reset the index  and rename the columns



In [ ]:
df.reset_index(inplace=True)
df.columns = [' '.join(col).strip() for col in df.columns.values]
df.rename({'rate PRVT':'Preventable mortality', 'rate TRT':'Treatable mortality'},inplace=True,axis=1)
df.head()

## GDP per capita



Go to this [page](https://ec.europa.eu/eurostat/data/database?node_code=nama_10_pc) to find the variable name for &ldquo;Main GDP aggregates per capita&rdquo;:



In [ ]:
df_n = eurostat.get_data_df('...')
df_n.rename(...)
df_n['country'] = ...
df_n.drop(np.arange(1975,2011),axis=1,inplace=True)
df_n.head()

### Question: make the following selections:



-   from the column `country` select EU countries
-   from `na_item` select &rsquo;B1GQ&rsquo;
-   from `unit` select `CP_EUR_HAB`
-   then drop the columns `unit, na_item, geo`



### answer



### The melt statement



The melt statement is easier now. Together with `year`, a column is uniquely identified by `country` only. Since we only have one variable here (GDP), we do not need to `unstack`.



### merge into dataframe



The final step is to [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) the GDP data with the dataframe `df` on mortality.



In [ ]:
df=df.merge(df_n,how='left',left_on=['country','year'], right_on=['country','year'])
df.head()

## Body mass index



-   go to: [https://ec.europa.eu/eurostat/databrowser/view/hlth_ehis_bm1i/default/table?lang=en](https://ec.europa.eu/eurostat/databrowser/view/hlth_ehis_bm1i/default/table?lang=en)
-   create a dataframe `df_n` for this data
-   make your own selections on the data
-   merge this data with the dataframe `df`



### merge into dataframe



## next lecture: Bayesian statistics



-   you have a coin and you want to determine the probability $p$ that it turns up head
-   experiment: you throw it once ($n=1$), twice ($n=2$), three times etc. all the way up to twenty times ($n=20$)
-   count the number of heads ($y$)
-   6 of these experiments ($n=1,2,3,4,5,20$) and outcomes($y = 1,1,2,2,3,14$)
-   after each experiment you calculate the posterior distribution for $p$



In [ ]:
y_values = [1,1,2,2,3,14]
n_values = [1,2,3,4,5,20]
posterior_list = []
for i in range(len(n_values)):
    with pm.Model() as binomial:
      p = pm.Uniform('p',0,1)
      y = pm.Binomial('y', p = p, observed = y_values[i], n = n_values[i])
      trace = pm.sample(tune=2000)
    print(i)
    posterior = az.from_pymc3(trace)
    posterior_list.append(posterior.posterior.p.values.flatten())

In [ ]:
posterior_list.append(np.arange(0,1.0001,0.0001))

We plot the posterior distributions:



In [ ]:
fig, axs = plt.subplots(2,3,figsize=(12,6))

for i in range(2):
    for j in range(3):
        sn.kdeplot(posterior_list[3*i+j],ax = axs[i,j])
        sn.kdeplot(posterior_list[3*i+j-1],ax = axs[i,j],linestyle='--')
        plt.gcf().get_axes()[3*i+j].set_xlim(0,1)
        axs[i,j].set_title('n = {}, y = {}'.format(n_values[3*i+j],y_values[3*i+j]))
fig.tight_layout();



The advantage of a posterior distribution is that it tells you what you expect it to tell you. To illustrate this, consider the posterior at $n=20$ which we plot now as cdf (cumulative distribution function):



In [ ]:
sn.kdeplot(posterior_list[-2],cumulative=True)
plt.ylabel('probability')
plt.xlabel('$p$');

This curve shows that with 40% probability it is the case that $p \leq 0.65$ (approximately).



### Question: calculate the probability that $p>0.8$ after throwing the coin 20 times (with $y=14$).



[hint: use the variable `posterior_list`.]



## Altair



In [ ]:
df = pd.read_csv('./data/health_data.csv')
df.describe()

You can open the file `gdp_mortality.html` in a web browser. You can add it to a website etc. You can use the slider to select a year, zoom in and out of the graph, hover over a point to see the country.



In [ ]:
df.rename({'year_x':'year'},inplace=True,axis=1)

select_year = alt.selection_single(
    name='Select', fields=['year'], init={'year': 2011},
    bind=alt.binding_range(min=2011, max=2017, step=1)
)

df['log gdp'] = np.log(df['GDP per capita'])
df['log mortality'] = np.log(df['Treatable mortality'])
df['Gender'] = df['sex'].replace({'F':'Female','M':'Male'})

figure = alt.Chart(df).mark_point(filled=True,size=50).encode(
    alt.X('log gdp',title='GDP per captita(in logs)',scale=alt.Scale(domain=[9,12])),
    alt.Y('log mortality',title='Treatable mortality (in logs)',scale=alt.Scale(domain=[4,6])),
    color='country',
    column='Gender',
    tooltip=['country']
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).add_selection(select_year).transform_filter(select_year).interactive()



figure.save('./figures/gdp_mortality.html')